# DocChat AI

In [36]:
import os
import openai
import langchain
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
import chromadb
import warnings
import json

warnings.filterwarnings('ignore')

## Extracting Text from PDF files

In [7]:
# Function to read pdf files
def read_pdf(directory_path):

    # Accessing the directory which contains the pdf file
    file_loader = PyPDFDirectoryLoader(directory_path)

    # Reading the document
    documents = file_loader.load()

    return documents

In [10]:
# Executing the function
doc = read_pdf('../src/files/')

In [11]:
doc

[Document(metadata={'source': '..\\src\\files\\Os_Segredos_da_Mente_Milionária.pdf', 'page': 0}, page_content=' \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n '),
 Document(metadata={'source': '..\\src\\files\\Os_Segredos_da_Mente_Milionária.pdf', 'page': 1}, page_content=' \nContra-capa \n \n Se as suas finanças andam na corda bamba, talvez esteja na hora de você \nrefletir sobre o que T . Harv  Eker chama de "o seu modelo de di nheiro" - um \nconjunto de crenças que cada um de nós ali menta desde a i nfância e que \nmolda o nosso destino financeiro, quase sempre nos levando para uma  situação \ndifícil. \n Neste li vro, Eker mostra como substi tuir uma mentali dade destruti va - que \nvocê talv ez nem perceba que tem - pelo s "arqui vos de ri queza", 17 modos de \npensar e agir que distinguem os ricos das demais pessoas. Alguns desses princípios \nfundamentais são: \n- Ou você controla o seu dinheiro ou ele controlará você. \n- O hábi to de admi nistrar as fi nanças é mai s i mporta

In [12]:
len(doc)

112

---

# Defining the LLM API

https://platform.openai.com/

https://platform.openai.com/api-keys

https://platform.openai.com/docs/quickstart?context=python

In [ ]:
config_path = '../config/secret_key.json'

with open(config_path, "r") as f:
    config = json.load(f)

## Embedding generator

OpenAIEmbeddings is a functionality that allows obtaining vector representations (embeddings) of text, which are useful for various natural language processing tasks, such as text similarity comparison, clustering, and classification.

In [23]:
# Creating the embedding generator
embedding_generator = OpenAIEmbeddings(api_key = config['secret_key'])

In [24]:
type(embedding_generator)

langchain_community.embeddings.openai.OpenAIEmbeddings

## Testing Embedding Vectors

In [25]:
vectors = embedding_generator.embed_query('What is the most important skill in the AI age?')

In [26]:
type(vectors)

list

In [28]:
vectors[0:10]

[-1.3786696116326466e-05,
 -0.006535670085572701,
 0.0141175768231183,
 -0.015627336466821143,
 0.010197499298800459,
 0.01435596002278955,
 -0.027493517231997736,
 0.02688431675652967,
 -0.010780213890366028,
 -0.045716582079977516]

## Defining the Vector Store

https://www.trychroma.com/

In [ ]:
# Defining the index name
index_name = 'docchat-index'

# Creating the vector store
index = Chroma.from_documents(doc, embedding_generator, collection_name=index_name)

type(index)

langchain_community.vectorstores.chroma.Chroma

In [38]:
# Defines the similarity search function
def similarity_search(query, k = 2):

    # Uses the 'similarity_search' method of the 'index' object to search for the 'k' results
    # most similar to the 'query'
    matching_results = index.similarity_search(query, k)

    # Returns the matching results of the similarity search
    return matching_results

## Building Personal Assistant App with LangChain

In [35]:
# Create LLM instance
llm = OpenAI(openai_api_key = config['secret_key'], temperature = 0.3)

https://python.langchain.com/docs/modules/chains/document/stuff

https://js.langchain.com/docs/use_cases/question_answering/

In [37]:
# Creates the chain for questions and answers in documents
chain = load_qa_chain(llm, chain_type = 'stuff')

In [39]:
# Defines the function to get the answer
def get_answer(query):

    # Calls the 'similarity_search' function with the given 'query' and stores the result in 'doc_search'
    doc_search = similarity_search(query)

    # Uses the 'chain' object to execute the run function and process the 'query' and the documents found,
    # storing the answer in 'response'
    response = chain.run(input_documents = doc_search, question = query)

    # Returns the answer obtained from the previous processing
    return response

---

In [40]:
# Question for the pdf file
prompt_1 = "Quais são os princípios de riqueza?"

# Get the answer
answer_1 = get_answer(prompt_1)

print(answer_1)

 Os princípios de riqueza mencionados no texto são: 
1. Crescer pessoalmente para superar problemas e obstáculos 
2. Concentrar-se nas metas e soluções, não nos problemas 
3. Mudar as raízes (mundo interior) para alterar os frutos (mundo exterior) 
4. Entender que a vida acontece em quatro quadrantes: físico, mental, emocional e espiritual.


In [41]:
# Question for the pdf file
prompt_2 = "Quais são algumas das possíveis desvantagens de ser rico ou tentar ser rico?"

# Get the answer
answer_2 = get_answer(prompt_2)

print(answer_2)

 Algumas possíveis desvantagens de ser rico ou tentar ser rico incluem o medo de perder tudo, a preocupação com a opinião dos outros, a alta carga tributária, o esforço e responsabilidade de administrar grandes quantidades de dinheiro, o risco de ser roubado ou ter seus filhos sequestrados, e a pressão para ajudar os outros financeiramente.
